# Geospatial Analytics

In this lab and exercise, we'll examine geospatial data related to bus stops and addresses and use it to answer some questions.

The data we'll be working with is available directly from [COTA](https://www.cota.com/data/) and the [City of Columbus](data-columbus.opendata.arcgis.com).  

As a reminder, press SHIFT+ENTER or press the *Run Cell* button to execute the code in a selected cell. For the lab, you do not need to add any code - just follow along.  For the exercise, you will have to write one line of code.

## Lab

In this lab, we'll use the data mentioned above to answer a question related to bus availability.  For a given address and a set of bus stops, we can calculate the distance to the nearest bus stop.  For a collection of addresses, we can calculate the distance to the nearest stop for each of them.  We can than determine what the greatest distance is from this set of calculations as a way of measuring how accessible public transportation is in an area.

Though the City of Columbus' Address dataset contains over 750,000 addresses for Franklin county and areas within 7 miles of the county, we'll work with a much smaller subset: the addresses in the [43222](https://www.google.com/maps/place/Columbus,+OH+43222/@39.96022,-83.0547451,14z/data=!3m1!4b1!4m5!3m4!1s0x88388f0a41e5927d:0x39750e5a4bb09747!8m2!3d39.9591613!4d-83.0334524) zip code.  This reduction is necessary given time and resource limitations.

### Libraries

To conduct this analysis, we'll make use of several third-party libraries:

- [geopandas](http://geopandas.org/): an extension of the pandas library for organizing and working with geospatial data.
- [haversine](https://github.com/mapado/haversine): a library that provides functionality to calculate the distance between two points on earth using the [haversine formula](https://en.wikipedia.org/wiki/Haversine_formula).
- [matplotlib](https://matplotlib.org/): a library used to generate plots; while we won't make use of the library directly, many other libraries rely on matplotlib to generate plots.
- [requests](http://docs.python-requests.org/en/master/): a library that provides a simple way making HTTP requests to get resources from the web.

As a first step, we'll make sure these libraries are installed. Some might already be installed in the environment in which the notebook is run.

In [ ]:
!pip install geopandas haversine matplotlib requests 

Now that the third-party libraries are installed, we have to use `import` to load them to make use of them.  In addition to the libraries listed above, we'll also load components from the [standard library](https://docs.python.org/3/library/index.html) - code included with a standard installation of Python that provides additional functionality:

- [io](https://docs.python.org/3/library/io.html): tools for working with [streams](https://en.wikipedia.org/wiki/Stream_%28computing%29) of data.
- [itertools](https://docs.python.org/3/library/itertools.html): a collection of utilities to support efficient iteration through collections.
- [zipfile](https://docs.python.org/3/library/zipfile.html): tools for working with zip files.

Additionally, we'll use the `from...import` syntax to import a specific component from the *matplotlib* library.

In [ ]:
import io
import itertools
import zipfile

import geopandas
import haversine
import requests
import shapely

from matplotlib import pyplot

As mentioned previously, matplotlib will be used to generate plots.  To provide a nicer integration with the notebook, we'll first indicate that we'd like generate plots to be show in the notebook itself and then set the figure size to have a value greater than the default.

In [ ]:
%matplotlib inline
pyplot.rcParams["figure.figsize"] = (10, 10)

### COTA Data

Now that we've loaded the necessary libraries, let's work with the COTA data.  The data for COTA bus stops is available at the URL below.  As a first step, we'll store this URL in a variable named *cota_data_url*.

In [ ]:
cota_data_url = "http://cota1974.maps.arcgis.com/sharing/rest/content/items/43e38988fab145e4934d6c6d7b3c2385/data"

If you were to open that URL in a browser, the site would return a zip file that would be saved to the computer.  Rather than download the zip file and process it outside the notebook, we can use Python and do the necessary work in the notebook.

First, we use the *Requests* library to make an [HTTP GET](https://www.w3schools.com/tags/ref_httpmethods.asp) request using the URL we specified above.  The response to our request is stored in the *cota_data* variable.

The response data is actually that of a zip file.  In order to interact with the data as a zip file, we need to first have Python convert it into a stream of binary data using the *io* library.  The resulting stream is stored in the *cota_data_stream* variable.

After converting the data into a binary stream, we can use the *zipfile* library to create an object that will allow us to interact with the data as a zip file, listing files, extracting data, etc. We'll store the zip file object in a variable named *cota_data_zip*.

In [ ]:
cota_data = requests.get(cota_data_url)
cota_data_stream = io.BytesIO(cota_data.content)
cota_data_zip = zipfile.ZipFile(cota_data_stream)

Now that we have an object that we can treat like a zip file, we can use the *namelist* function to see all the files contained in the zip file itself.

In [ ]:
cota_data_zip.namelist()

These are [common](http://gisgeography.com/arcgis-shapefile-files-types-extensions/) [ESRI ArcGIS](http://www.esri.com/arcgis/about-arcgis) file formats that are used to describe geographic features. In this case, we have two sets of data: one corresponding to bus routes and one corresponding to bus stops. We're interested in the bus stops.  

In order to use the data, we first have to extract the files from the zip file. To do this, we an use the *extractall* method on *ZipFile* objects.

In [ ]:
cota_data_zip.extractall()

Now that files have been extracted from the zip file, we can load the bus stop data using the *Geopanadas* library and store it in a variable named *cota_stops* and see how many records are included in the dataset.

In [ ]:
cota_stops = geopandas.read_file('Stops_201709.shp')
len(cota_stops)

Now that the data has been loaded, we can begin examining it.  *Geopandas* extends the *pandas* library and the *pandas* library makes use of a tabular data structure knowns as a *[GeoDataFrame](http://geopandas.org/data_structures.html#geodataframe)*.  We can examine the first five records of the table created from the bus stop data using the *head* method.

In [ ]:
cota_stops.head(5)

From the first five rows, we can see examples of data stored in each column.  Notice that not all the columns are displayed due to the number of columns available.  To see data for all the columns, we can either increase the number of columns that are displayed by default or examine an individual record using the *iloc* object associated with a *GeoDataFrame*.  The *iloc* object can be treated like a collection of items so we can access individual elements using the bracket notation and specifying the index of the element we're interested in.  We'll look at the first row.

In [ ]:
cota_stops.iloc[0]

One important field to note is *geometry*.  Each bus stop can be considered in terms of a point on a map. The coordinates of each point are given by the values displayed in the *geometry* field.  For our analysis, it would be nice if these coordinates corresponded to longitude and latitude but it doesn't appear to be the case.  

A common issue with mapping is choosing how to represent the surface of a three-dimensional object, such as the earth, in a two-dimensional plane, such as a map.  This problem has led to the development of different solutions based on different [projections](https://en.wikipedia.org/wiki/Map_projection) and [coordinate systems](https://en.wikipedia.org/wiki/Geodetic_datum).  To see the projection being used with this data and to understand the meaning of the values in *geometry*, we can use the *csr* property of a *GeoDataFrame* to see which coordinate reference system it uses.

In [ ]:
cota_stops.crs

The result indicates that the data makes use of the [Lambert conformal conic projection](https://en.wikipedia.org/wiki/Lambert_conformal_conic_projection) and the [North American Datum of 1983](https://en.wikipedia.org/wiki/North_American_Datum#North_American_Datum_of_1983) coordinate system.

We'd like to use the [WGS 84](https://en.wikipedia.org/wiki/World_Geodetic_System) system, also known as [EPSG:4326](http://epsg.io/4326), a system that commonly uses an [equirectangular projection](https://en.wikipedia.org/wiki/Equirectangular_projection) for 2D representations and the standard latitude/longitude coordinate system.  This commonly used system is used in a variety of applications including [GPS](https://en.wikipedia.org/wiki/Global_Positioning_System).  

To change the coordinate reference system, we can use the *to_crs* function and specify the properties of the new system.  In this case, we'll use `{'init': 'epsg:4326'}` to specify EPSG:4362.

In [ ]:
cota_stops = cota_stops.to_crs({'init': 'epsg:4326'})
cota_stops.crs

We can also look at a specific record in the GeoDataFram to confirm that geometry point data now represents longitude-latitude pairs.

In [ ]:
cota_stops.iloc[0]

Note that the geometry data now corresponds to the values in the *Lon* and *Lat* fields.

Before continuing, let's discard bus stops that are not in service.  We can do this by filtering the dataset based on the value in the *InService* field.

In [ ]:
cota_stops = cota_stops[cota_stops.InService == 1]
len(cota_stops)

Only ten stops are marked as not being in service.

One way to get a sense of the data in the GeoDataFrame is to plot it.  The *GeoDataFrame* *plot* method can be used.  Since the data contains point data, the plot will consist of a collection of points.  For this plot, we'll make the point markers blue.

In [ ]:
cota_stops.plot(color='b')

Looking at the plot, we can see that most bus stops correspond to major roadways in the Columbus area. 

### Address Data

Now that we've transformed the bus stop data into the state we'd like, let's begin working with the address data. 

We'll be working with a subset of the entire address dataset restricted to the 43222 zip code. To start, we can load the data directly from the file.  Note that the data is stored in the [GeoJSON](http://geojson.org/) format, making using of JSON but with a specific, geodata-related structure. Once we load the data, we can see the number of addresses contained in the data.

In [ ]:
addresses = geopandas.read_file("/home/nbuser/43222.geojson")
len(addresses)

There are nearly 3,000 addressess contained in the dataset. Let's see what the first few records looks like and all the fields and values for the first record.

In [ ]:
addresses.head(5)

In [ ]:
addresses.iloc[0]

Notice that the address data includes a *geometry* field that appears to contain point data in terms of longitude and latitude.  Most geospatial datasets will include a *geometry* field that includes *point*, *line*, or *polygon* data or a representation of square cells.  To see the details of the coordinate system being used, we can use the *crs* property again.

In [ ]:
addresses.crs

This is the system we'd like to work in so we don't need to transform the data before working with it. 

Let's look at the address data in relation to the bus stop data.  To plot both of them together, we first create *matplotlib* *figure* and *axes* objects.  The *axes* object will be stored in the *ax* variable and used with the *GeoDataFrame* *plot* method.  We'll plot bus stops with blue markers and addresses with red markers.

In [ ]:
fig, ax = pyplot.subplots(1, 1)
cota_stops.plot(ax=ax, color='b')
addresses.plot(ax=ax, color='r')

Notice that the area that includes address points is much smaller than the area that contains all the bus stops.  We can filter the bus stop data by specifying bounds on the longitude and latitude.

In [ ]:
cota_stops = cota_stops[(cota_stops.geometry.x >= -83.1) &
                                 (cota_stops.geometry.x <= -83.0) &
                                 (cota_stops.geometry.y >= 39.9) &
                                 (cota_stops.geometry.y <= 40.0)]
len(cota_stops)

We've reduced the number of bus stops to just over 500.  Let's update our plot of addresses and bus stops.

In [ ]:
fig, ax = pyplot.subplots(1, 1)
addresses.plot(ax=ax, color='r')
cota_stops.plot(ax=ax, color='b')

### Greatest Minimum Distance

With the data loaded and the dataset pared down a bit, we can now consider the question we're trying to answer.  First, let's see how we can calculate the physical distance in miles between a bus stop and an address. 

In [ ]:
a_stop = cota_stops.iloc[0].geometry
an_address = addresses.iloc[0].geometry
haversine.haversine((a_stop.y, a_stop.x), (an_address.y, an_address.x), miles=True)

To use the *haversine* function, we had to rewrite the coordinates of each point as latitude/longitude pairs rather than longitude/latitude pairs.  This is a bit cumbersome so we can write our own function to calculate the distance between two points as they are stored in the datasets.

In [ ]:
def distance(point_1, point_2):
    lat_lon_1 = (point_1.y, point_1.x)
    lat_lon_2 = (point_2.y, point_2.x)
    return haversine.haversine(lat_lon_1, lat_lon_2, miles=True)

Before we look at all addresses, let's think about how we'd calculate the distance to the nearest bus stop for one address.  We'll start by choosing one address to work with, which we will store in the *address_point* variable.  Next, we can create a variable to store the smallest distance between the address and a bus stop, let's call it *minimum_stop_distance*, and set to the distance between the address and the first bus stop. To find the smallest distances, we examine each bus stop and calculate the distance between it and the address.  If the calculated distance is smaller than *minimum_stop_distance* we'll replace the value of *minimum_stop_distance* with the new value; if not, we'll continue on to the next address.  After examining all the bus stops, we'll display the value of *minimum_stop_distance*.

In [ ]:
address_point = addresses.iloc[0].geometry
minimum_stop_distance = distance(address_point, cota_stops.iloc[0].geometry)

for stop_point in cota_stops.geometry:
    stop_distance = distance(address_point, stop_point)
    if stop_distance < minimum_stop_distance:
        minimum_stop_distance = stop_distance
        
print("Miniumn stop distance:", minimum_stop_distance)

The result indicates that the nearest bus stop to the address we're working with is about one tenth of a mile away.  

Let's extend our code to look at all addresses, one at a time.  For each address, we'll calculate the distance to the nearest bus stop just like we did above.  Now that we're looking at all addresses, we'll also keep track of *greatest_minimum_distance*. We'll set *greatest_minimum_distance* to 0 initially.  For each address, we'll compare its distance to the nearest bus stop to *greatest_minimum_distance*.  If the distance to the nearest bus stop is greater than the value of *greatest_minimum_distance*, we'll replace the value of *greatest_minimum_distance* with the new value; if not, we'll continue to the next address. After working through all addresses, we'll display the value of *greatest_minimum_distance*.

The code below includes extra code to display the progress as it works through the calculations. The percentage is determined based on the index of the current address being processed divided by the total number of addresses.

In [ ]:
number_of_addresses = len(addresses)
greatest_minimum_distance = 0
for index, address_point in enumerate(addresses.geometry):
    
    # display progress
    if index % 100 == 0:
        print("Percent complete: ", index/number_of_addresses * 100)
    
    minimum_stop_distance = distance(address_point, cota_stops.iloc[0].geometry)
    for stop_point in cota_stops.geometry:
        stop_distance = distance(address_point, stop_point)
        if stop_distance < minimum_stop_distance:
            minimum_stop_distance = stop_distance
    
    if minimum_stop_distance > greatest_minimum_distance:
        greatest_minimum_distance = minimum_stop_distance
print("Greatest miniumn distance:", greatest_minimum_distance)

From the result we can see that the greatest distance between an address in the 43222 zip code and its nearest bus stop is about half a mile. 

## Exercise
Calculate the mean distance to the nearest stop for all addresses in 43222.

**Hint**: Most of the code used to calculate the greatest minimum distance can be reused. Here's an updated form of the code above with some new variable names.

```python
number_of_addresses = len(addresses)
total_minimum_distance = 0
for index, address_point in enumerate(addresses.geometry):
    
    # display progress
    if index % 100 == 0:
        print("Percent complete: ", index/number_of_addresses * 100)
    
    minimum_stop_distance = distance(address_point, cota_stops.iloc[0].geometry)
    for stop_point in cota_stops.geometry:
        stop_distance = distance(address_point, stop_point)
        if stop_distance < minimum_stop_distance:
            minimum_stop_distance = stop_distance
    total_minimum_distance = total_minimum_distance + minimum_stop_distance

# MEAN CALCULATION CODE HERE

print("Mean minimumn distance:", mean_minimum_distance)
```

Replace "`# MEAN CALCULATION CODE HERE`" with the code needed to calculate `mean_minimum_distance` from `total_minimum_distance` and `number_of_addresses`.

Recall that the mean of a set of values is the sum of the values divided by the number of values.